In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
data = pd.read_csv('/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data/train-dataset.csv',usecols=['itemId','label'])

In [ ]:
target = data.pop('label')
dataset = tf.data.Dataset.from_tensor_slices((data.values, target.values))

In [ ]:
val_data = dataset.take(1000)

In [ ]:
train_data = dataset.skip(1000)

In [ ]:
train_data = train_data.shuffle(1000).batch(64)

In [ ]:
inputs = {
    'itemId':tf.keras.layers.Input(name='itemId', shape=(), dtype='int32'),
}

In [ ]:
data.itemId.max()

In [ ]:
item_features = tf.keras.layers.Embedding(10124082, 64 )(inputs['itemId'])

In [ ]:
item_features = tf.keras.layers.Dense(32,activation='relu')(item_features)


In [ ]:
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(item_features)
outputs

In [ ]:
model = tf.keras.Model(inputs,outputs)

In [ ]:
METRICS = [
      # tf.keras.metrics.TruePositives(name='tp'),
      # tf.keras.metrics.FalsePositives(name='fp'),
      # tf.keras.metrics.TrueNegatives(name='tn'),
      # tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      # tf.keras.metrics.Precision(name='precision'),
      # tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=METRICS
)

In [ ]:
model.fit(train_data,epochs=1)

In [ ]:
model.save('./load_dataset.h5')

In [ ]:
model.input

In [ ]:
model.get_layer('embedding').output

In [ ]:
item_layer_model = tf.keras.Model(inputs,outputs=model.get_layer('embedding').output)

In [ ]:
item_layer_model(data.head(1)['itemId'].values)

In [ ]:
item_layer_model.predict(data.head(1)['itemId'].values)

In [ ]:
item_layer_model

In [ ]:
import numpy as np
from tqdm import tqdm
item_embeddings = {}
for one_data in tqdm(data.itemId.unique()):
    # itemId = np.reshape(row['itemId'],(-1,1))
    item_embedding  = item_layer_model(np.array(one_data))
    item_embeddings[one_data] = item_embedding


In [ ]:
#变成csv
data_csv = pd.DataFrame(columns=['itemId','Embedding'])

data_csv['itemId'] = item_embeddings.keys()
for i,one_data in data_csv.values:
    one_data['Embedding'] = item_embeddings.values()
    # one_data = np.array(one_data)
# data_csv['Embedding'] = np.array(item_embeddings.values())



In [ ]:
i = 0
for value  in item_embeddings.values():
    data_csv['Embedding'][i] = np.array(value)
    i = i+1

In [ ]:
data_csv['Embedding'][0]

In [ ]:
import faiss

In [ ]:
item_embedding_array_list = []
for x in data_csv['Embedding'].values:
    item_embedding_array_list.append(x)
item_embedding_array_list = np.array(item_embedding_array_list).astype(np.float32)

In [ ]:
item_embedding_array_list

In [ ]:
import faiss
dimension = 64
#建立索引
index = faiss.IndexFlatL2(dimension)
#将向量加进faiss库中
index.add(item_embedding_array_list)

In [ ]:
top_k = 5

#D为距离，I为索引
D,I = index.search(item_embedding_array_list[1:5],top_k)

In [ ]:
D

In [ ]:
I